<a href="https://colab.research.google.com/github/mnansary/pyF2O/blob/master/colab_gen_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   Check TF version
*   mount google drive
*   TPU check
*   Change Working Directory to cloned dir 

NOTE: **git clone the repo before running this notebook**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:

# tpu check
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', TPU_ADDRESS)

  with tf.Session(TPU_ADDRESS) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)


TPU address is grpc://10.20.28.90:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 16201179908277930962),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10658802392888327259),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11151935654971009028),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10265847771783731301),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13564940274577102710),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5629598095619318250),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 8371747072504508936),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5415472860107639727),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 160476045866

In [3]:
cd /content/gdrive/My\ Drive/PROJECTS/F2O/pyF2O/

/content/gdrive/My Drive/PROJECTS/F2O/pyF2O


# GCS specific task 
* **auth user**
* **save** and **upload** credentials to **tpu**
* set project information


In [0]:
# auth user for cloud SDK
from google.colab import auth
auth.authenticate_user()

In [5]:
# Save credentials
import json
SERVICE_KEY_PATH='/content/adc.json' # @param
# Upload credentials to TPU.
with tf.Session(TPU_ADDRESS) as sess:    
    with open(SERVICE_KEY_PATH, 'r') as f:
        auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
# set service_account
JSON_DATA=json.load(open(SERVICE_KEY_PATH))
SERVICE_ACCOUNT=str(JSON_DATA['client_id']).split('.')[0]
print('Service Account:',SERVICE_ACCOUNT)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Service Account: 32555940559


#### SET PROJECT INFORMATION 

In [6]:
PROJECT_ID    ='f2oitpu'               # @param 
BUCKET        ='f2odata'               # @param 

# LIST FILES
TFRECORDS_DIR= 'gs://{}/{}/'.format(BUCKET,'tfrecord')
MODEL_DIR    = 'gs://{}/{}/'.format(BUCKET,'model_dir')
# change TFRECORDS_DIR specific to structre
#
!gcloud config set project {PROJECT_ID}
!gsutil ls {TFRECORDS_DIR}
!gsutil ls gs://{BUCKET}


Updated property [core/project].
gs://f2odata/tfrecord/eval/
gs://f2odata/tfrecord/test/
gs://f2odata/tfrecord/train/
gs://f2odata/model_dir/
gs://f2odata/tfrecord/


# pix2pix
* set **FLAGS** 
* define **model function**
* define **input function**
* define **config**
* define **estimator**

#### FLAGS AND PARAMS

In [7]:
class FLAGS:
    IMAGE_DIM       = 256       # @param
    NB_CHANNELS     = 3         # @param
    BATCH_SIZE      = 128       # @param
    SHUFFLE_BUFFER  = 1000      # @param    
    NB_TOTAL_DATA   = 16128     # @param
    NB_EVAL_DATA    = 3200      # @param
    ITERATIONS      = 2       # @param
    LEARNING_RATE   = 0.0002    # @param

MODEL_EXPORT_NAME = 'f20-pix2pix' # @param
STEPS           = FLAGS.NB_TOTAL_DATA // FLAGS.BATCH_SIZE 
print('Num of Steps:{}'.format(STEPS))


Num of Steps:126


#### Model Function

In [0]:
from F2O.pix2pix import loss_fn
def model_fn(features, labels, mode, params):
    if (mode != tf.estimator.ModeKeys.PREDICT):
        loss = loss_fn(features, labels)
        learning_rate = tf.train.exponential_decay(FLAGS.LEARNING_RATE,tf.train.get_global_step(),decay_steps=100000,decay_rate=0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
        train_op=optimizer.minimize(loss,tf.train.get_global_step())
        return tf.contrib.tpu.TPUEstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    else:
        predictions=generator_fn(features)
        return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)   

#### Data Input Functions
 

In [9]:
from google.cloud import storage
from functools import partial

client = storage.Client(PROJECT_ID)
# get bucket from the project
bucket=client.get_bucket(BUCKET)
print(bucket)

def data_input_fn(FLAGS,mode,params): 
    
    def _parser(example):
        feature ={  'image'  : tf.io.FixedLenFeature([],tf.string) ,
                    'target' : tf.io.FixedLenFeature([],tf.string)
        }    
        parsed_example=tf.io.parse_single_example(example,feature)
        image_raw=parsed_example['image']
        image=tf.image.decode_png(image_raw,channels=FLAGS.NB_CHANNELS)
        image=tf.cast(image,tf.float32)/255.0
        image=tf.reshape(image,(FLAGS.IMAGE_DIM,FLAGS.IMAGE_DIM,FLAGS.NB_CHANNELS))
        
        target_raw=parsed_example['target']
        target=tf.image.decode_png(target_raw,channels=FLAGS.NB_CHANNELS)
        target=tf.cast(target,tf.float32)/255.0
        target=tf.reshape(target,(FLAGS.IMAGE_DIM,FLAGS.IMAGE_DIM,FLAGS.NB_CHANNELS))
        if mode=='test':
          return image
        else:
          return image,target

    dataset = tf.data.TFRecordDataset([os.path.join('gs://{}/'.format(BUCKET), f.name) for f in bucket.list_blobs(prefix='tfrecord/{}'.format(mode))])
    if mode!='test':
      dataset = dataset.cache()
      dataset = dataset.shuffle(FLAGS.SHUFFLE_BUFFER,reshuffle_each_iteration=True)
    dataset = dataset.map(_parser)
    if mode!='test':
      dataset = dataset.repeat()
      dataset = dataset.batch(FLAGS.BATCH_SIZE,drop_remainder=True)
      dataset = dataset.prefetch(-1) # autotune
    else:
      dataset = dataset.repeat(1)
      dataset = dataset.batch(FLAGS.BATCH_SIZE)
      
    return dataset

train_input_fn = partial(data_input_fn,FLAGS=FLAGS,mode='train')
test_input_fn  = partial(data_input_fn,FLAGS=FLAGS,mode='test' )


<Bucket: f2odata>


#### TPU config and estimator

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)
 
TPU = tf.contrib.cluster_resolver.TPUClusterResolver()
 
config = tf.contrib.tpu.RunConfig(
                                   cluster=TPU,
                                   model_dir=MODEL_DIR,
                                   tpu_config=tf.contrib.tpu.TPUConfig(STEPS)
                                   )
estimator = tf.contrib.tpu.TPUEstimator(
                                        model_fn=model_fn,
                                        model_dir=MODEL_DIR,
                                        train_batch_size=FLAGS.BATCH_SIZE,
                                        predict_batch_size=FLAGS.BATCH_SIZE,
                                        config=config,
                                        use_tpu=True,
                                        export_to_tpu=False
                                        )  
# we want an exported model for CPU/GPU inference 
estimator.train(train_input_fn, steps=STEPS*FLAGS.ITERATIONS)
predictions=estimator.predict(input_fn=test_input_fn)

INFO:tensorflow:Using config: {'_model_dir': 'gs://f2odata/model_dir/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.20.28.90:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3baa302a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.20.28.90:8470', '_evaluation_master': 'grpc://10.20.28.90:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_

In [11]:
for pred in predictions:
    print(pred) 

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Error recorded from prediction_loop: in converted code:

    /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py:3968 _map_fn  *
        padding_mask, features, labels = (
    /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py:4061 pad_single_tensor  *
        padded_tensor = array_ops.pad(tensor, padding)
    /tmp/tmpbpf8pxel.py:44 nest_pad
        retval__2 = nest_pad_scope.mark_return_value(ag__.converted_call(nest.map_structure, nest_pad_scope.callopts, (pad_single_tensor, tensor_or_dict), None, nest_pad_scope))
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/nest.py:536 map_structure
        structure[0], [func(*x) for x in entries],
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/array_ops.py:2840 pad
        result = gen_array_ops.pad(tensor, paddings, name=name)
    /usr/local/lib/python3.6/d

ValueError: ignored

!gsutil -m cp -R gs://f2odata/model_dir /content/gdrive/My\ Drive/PROJECTS/F2O/model_dir